In [ ]:
import sys
sys.path.append('..')

In [ ]:
# Load terminology
from base.load_hpo import hpo

In [ ]:
# Load datasets
from base.load_dataset import load_train_dataset, load_val_dataset
train_dataset = load_train_dataset()
val_dataset = load_val_dataset()

In [ ]:
# Load OpenAI client
from config.openai_config import openai_api_key
from prompting.openai_client import FinetuningOpenAIClient
openai_client = FinetuningOpenAIClient(openai_api_key, model='gpt-3.5-turbo-0613')

In [ ]:
# Load prompting
from prompting.generate_messages import get_openai_finetuning_messages
from prompting.prompts import *

# Create datasets for OpenAI API
train_messages = get_openai_finetuning_messages(train_dataset, hpo=hpo, system_message=SYSTEM_MESSAGE,
                                                user_message_wrapper=USER_MESSAGE_WRAPPER,
                                                assistant_message_table_header=ASSISTANT_MESSAGE_TABLE_HEADER,
                                                filename='train.json')
val_messages = get_openai_finetuning_messages(val_dataset, hpo=hpo, system_message=SYSTEM_MESSAGE,
                                              user_message_wrapper=USER_MESSAGE_WRAPPER,
                                              assistant_message_table_header=ASSISTANT_MESSAGE_TABLE_HEADER,
                                              filename='val.json')

In [ ]:
# Upload datasets to OpenAI API
import os
from config.config import FINE_TUNE_DIR
openai_client.upload_train_file(os.path.join(FINE_TUNE_DIR, 'train.json'))
openai_client.upload_val_file(os.path.join(FINE_TUNE_DIR, 'val.json'))

In [ ]:
# Fine tune a model
openai_client.model = 'gpt-3.5-turbo-0613'
openai_client.finetune()

In [ ]:
# Retrieve job
openai_client.retrieve_finetune_job()

In [ ]:
# Retrieve finetuned model name if the job succeeded
openai_client.get_finetuned_model()

In [ ]:
# Switch to using finetuned model for future requests
openai_client.use_finetuned_model()

In [ ]:
# Example request
item = val_messages[0]
response = openai_client.get_response(item['messages'])
print(response)